In [2]:
# Multiple Outputs
import numpy as np
import os
import cv2

## Dataset iterator

In [3]:
FACES_PATH = '../data/face_detection/faces/'

In [4]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


In [5]:
# Set up some global variables
USE_GPU = False

if USE_GPU:
    device = '/device:GPU:0'
else:
    device = '/cpu:0'

# Constant to control how often we print when training models
print_every = 100

print('Using device: ', device)

Using device:  /cpu:0


### Read training dataset

In [8]:
training_size = 500

def read_pos_images():
    #Read positive images:
    path, __, filenames = next(os.walk(FACES_PATH+'pos_train/'))
    file_count = training_size #len(filenames)
    images = np.empty([0,12,3])
    for i in range(file_count):
        j=i+1
        img=cv2.imread(f"{path}{j}.bmp")
        images=np.append(images,img,axis=0)
    #Create list of probabilities:
    prob=[]
    for i in range(file_count):
        prob.append([[[0.0,1.0]]])
    #Create list of coordinates:
    coordinates=[]
    file = open(FACES_PATH+'coordinates.txt','r')
    lines = file.readlines()
    lines = [line[:-1] for line in lines]
    idx=[1,0,3,2]
    for line in lines:
        line = line.split(" ")
        line = line[1]
        line=line[1:-1]
        line = line.split(",")
        #Transpose coordinates
        x=0
        nline=[]
        for i in idx:
            nline.append(line[i])
            x=x+1
        line=[[[float(c) for c in nline]]]
        coordinates.append(line)
    #Return images, probs, and coordinates
    return images, prob, coordinates

def read_neg_images():
    #Read negative images:
    path, __, filenames = next(os.walk(FACES_PATH+'neg_train/'))
    file_count = training_size #len(filenames)
    images = np.empty([0,12,3])
    for i in range(file_count):
        j=i+1
        img=cv2.imread(f"{path}{j}.bmp")
        images=np.append(images,img,axis=0)
    #Create list of probabilities:
    prob=[]
    for i in range(file_count):
        prob.append([[[1.0,0.0]]])
    #Create list of coordinates:
    coordinates=[]
    for i in range(file_count):
        coordinates.append([[[0.0,0.0,0.0,0.0]]])
    #Return images, prob, coordinates
    return images, prob, coordinates

#Read in all images, probabilities, and coordinates
pimages, pprob, pcoordinates = read_pos_images()
nimages, nprob, ncoordinates = read_neg_images()
o_images=np.append(pimages,nimages,axis=0)
o_images=np.reshape(o_images,(-1,12,12,3))
o_prob=pprob+nprob
o_coordinates=pcoordinates+ncoordinates

#Shuffle them up using an index
idx=np.arange(len(o_prob))
np.random.shuffle(idx)
images=np.empty_like(o_images)
c=0
for i in idx:
    images[c]=o_images[i]
    c=c+1
#images=(np.float32)(images-127.5)/128.0
images=(np.float32)(images)/255

#images = np.transpose(images, (0, 2, 1, 3)) #Transpose images
prob=[]
for i in idx:
    prob.append(o_prob[i])
coordinates=[]
for i in idx:
    coordinates.append(o_coordinates[i])

In [9]:
print('X_train , Image batch shape ', images.shape)
print('y_train , Classification ground true batch shape ' ,np.array(prob).shape)
print('y_train , Coordinates ground true batch shape ', np.array(coordinates).shape)

X_train , Image batch shape  (1000, 12, 12, 3)
y_train , Classification ground true batch shape  (1000, 1, 1, 2)
y_train , Coordinates ground true batch shape  (1000, 1, 1, 4)


## Create X_data for train and validation

In [10]:
X_data = images

In [11]:
print('X_data shape',X_data.shape)

X_data shape (1000, 12, 12, 3)


In [12]:
del images

## Create "y_data" for train and validation

In [31]:
y_data = np.concatenate((np.array(prob), np.array(coordinates)), axis=3)

In [32]:
print('y_data shape',y_data.shape)

y_data shape (1000, 1, 1, 6)


In [33]:
print('y_data Classification shape', y_data[:,:,:,:2].shape)
print('y_data Coordinate shape',y_data[:,:,:,2:].shape)

y_data Classification shape (1000, 1, 1, 2)
y_data Coordinate shape (1000, 1, 1, 4)


## Divide dataset to "train', "val" and "test"

In [16]:
def load_data(X, y, training_prec = 0.7, val_prec = 0.1, test_prec = 0.2):
        data_length = len(X)
        num_training = np.int(data_length * training_prec)
        num_validation = np.int(data_length * val_prec)
        
        mask = range(num_training)
        X_train = X[mask]
        y_train = y[mask]
        mask = range(num_training, num_training + num_validation)
        X_val = X[mask]
        y_val = y[mask]
        mask = range(num_training + num_validation, data_length)
        X_test = X[mask]
        y_test = y[mask]
        
        return X_train, y_train, X_val, y_val, X_test, y_test


In [17]:
X_train, y_train, X_val, y_val, X_test, y_test = load_data(X_data, y_data)
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (700, 12, 12, 3)
Train labels shape:  (700, 1, 1, 6) float64
Validation data shape:  (100, 12, 12, 3)
Validation labels shape:  (100, 1, 1, 6)
Test data shape:  (200, 12, 12, 3)
Test labels shape:  (200, 1, 1, 6)


In [18]:
train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

## Build P-Net Keras model

In [116]:
import tensorflow as tf
from keras.utils import plot_model
from keras.models import Model
from keras.layers import MaxPooling2D, Conv2D, Input, Layer, Concatenate, concatenate
from keras.layers.advanced_activations import PReLU
#from keras.layers.wrappers import TimeDistributed

In [122]:
def PNet():
    
    #initializer = tf.keras.initializers.VarianceScaling(scale=2.0)
    
    #input layer
    visible = Input(shape=(12,12,3))
    conv1 = Conv2D(10, kernel_size=(3,3))(visible)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    prelu1 = PReLU(alpha_initializer ='zero', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(pool1)
    
    conv2 = Conv2D(16, kernel_size=(3,3))(prelu1)
    prelu2 = PReLU(alpha_initializer='zero', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(conv2)
    
    conv3 = Conv2D(32, kernel_size=(3,3),)(conv2)
    prelu3 = PReLU(alpha_initializer='zero', alpha_regularizer=None, alpha_constraint=None, shared_axes=[1,2])(conv3)
    
    output1 = Conv2D(2, kernel_size=(1,1), activation='softmax')(conv3)
    output2 = Conv2D(4, kernel_size=(1,1))(conv3)
    
    #output = tf.concat((output1, output2), axis=3)
    output = concatenate([output1, output2], axis=-1)
    
    model = Model(inputs=visible, outputs=output)
                  
    #compute the loss function over the classification and over bounding box 
    classification_loss = tf.keras.losses.BinaryCrossentropy()
    bbox_loss = tf.keras.losses.MeanSquaredError() 
    
    # Use the model function to build the forward pass.

    #Define custom loss
    def custom_loss(calss_score, bbox_score):

        # Create a loss function for P-Net
        def loss(y_true,y_pred):
            prediction_loss = classification_loss(y_true[:,:,:,:2], calss_score)
            coordinate_loss = bbox_loss(y_true[:,:,:,2:], bbox_score)
            return prediction_loss + 0.5 * coordinate_loss * y_true[:,:,:,1]

   
        # Return a function
        return loss
    
    learning_rate = 1e-3
    adam = tf.keras.optimizers.Adam(learning_rate)
    model.compile(optimizer=adam, 
                  loss = custom_loss(output1, output2),
                  metrics=['accuracy'])
    # summarize layers
    print(model.summary())
    # plot graph
    plot_model(model, to_file='multiple_outputs.png')
    
    return model

In [123]:
model = PNet()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_41 (InputLayer)           (None, 12, 12, 3)    0                                            
__________________________________________________________________________________________________
conv2d_201 (Conv2D)             (None, 10, 10, 10)   280         input_41[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_41 (MaxPooling2D) (None, 5, 5, 10)     0           conv2d_201[0][0]                 
__________________________________________________________________________________________________
p_re_lu_121 (PReLU)             (None, 5, 5, 10)     10          max_pooling2d_41[0][0]           
____________________________________________________________________________________________

In [124]:
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_val, y_val))

Train on 700 samples, validate on 100 samples
Epoch 1/10
700/700 [==============================] - 1s 2ms/step - loss: 0.7432 - accuracy: 0.4300 - val_loss: 0.6834 - val_accuracy: 0.1100
Epoch 2/10
700/700 [==============================] - 0s 217us/step - loss: 0.6587 - accuracy: 0.0357 - val_loss: 0.6455 - val_accuracy: 0.0800
Epoch 3/10
700/700 [==============================] - 0s 221us/step - loss: 0.6050 - accuracy: 0.0643 - val_loss: 0.6019 - val_accuracy: 0.2100
Epoch 4/10
700/700 [==============================] - 0s 236us/step - loss: 0.5489 - accuracy: 0.0757 - val_loss: 0.5603 - val_accuracy: 0.1700
Epoch 5/10
700/700 [==============================] - 0s 170us/step - loss: 0.4963 - accuracy: 0.0671 - val_loss: 0.5329 - val_accuracy: 0.2100
Epoch 6/10
700/700 [==============================] - 0s 224us/step - loss: 0.4557 - accuracy: 0.1357 - val_loss: 0.4849 - val_accuracy: 0.2700
Epoch 7/10
700/700 [==============================] - 0s 211us/step - loss: 0.4062 - accurac

### Test the PNet  to ensure that the implementation does not crash and produces outputs of the expected shape.
Pnet will output are:
1. Face classification,  size (batch,1,1,2) for 2 calss classification, "Face", and "Not face"
2. Bounding box  (batch,1,1,4) for 4 boundind box corrdinates (x,y,w,h)